In [ ]:
!pip install pytorch-gradcam
!pip install opencv-python

In [ ]:
import torch
from torchvision import models
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Classification(torch.nn.Module):

    def __init__(self):
        super(Classification, self).__init__()
        self.backbone = models.resnet50()
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 2)
       
    def forward(self, x):
        out = self.backbone(x)

# Instantiate your model
model = Classification()
        
_ = model.load_state_dict(torch.load('./resnet50_pretrain.pth') , strict=False)

In [ ]:
device = torch.device("cuda")

In [ ]:
import os
from sklearn.model_selection import train_test_split

# Directory containing all images
image_dir = './Dataset/'

# Read all the file names
all_images = os.listdir(image_dir)

# Split the file names into train and test
train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)

print(f"Total {len(all_images)} images split into {len(train_images)} training and {len(test_images)} test images.")

In [ ]:
import shutil

# Directories for train and test images
train_dir = './train_directory/'
test_dir = './test_directory/'

# Move the images
for img in train_images:
    shutil.move(image_dir + img, train_dir + img)

for img in test_images:
    shutil.move(image_dir + img, test_dir + img)

In [ ]:
import os
import torch
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.file_list = os.listdir(directory)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        image_path = os.path.join(self.directory, file_name)

        # Load image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations to the image
        if self.transform:
            image = self.transform(image)

        # Set label based on filename
        if 'abnormal' in file_name:
            label = 1
        else:
            label = 0

        return image, label


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define your datasets
train_dataset = CustomDataset(train_dir, transform=transform)


# Define your dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CustomDataset(test_dir, transform=transform)


# Define your validation dataloaders
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)


# Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#
best_f1 = 0.0

num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()  # set the model to training mode
    for i, (inputs, labels) in enumerate(train_dataloader):
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Train Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    # Validation
    model.eval()  # set the model to evaluation mode
    with torch.no_grad():
        all_labels = []
        all_predictions = []
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels)
            all_predictions.extend(predicted)

        # Convert lists to tensors for metrics calculation
        all_labels = torch.stack(all_labels).to('cpu')
        all_predictions = torch.stack(all_predictions).to('cpu')

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')
        f1 = f1_score(all_labels, all_predictions, average='weighted')

        print(f'Val Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), 'best_model.pth')
            torch.jit.save(torch.jit.script(model), 'cls_model.pt')


In [ ]:
# Get a batch of data
data, _ = next(iter(train_dataloader))

# Select the first image from the batch
data = data[0].unsqueeze(0)

# Ensure the data is on the right device
data = data.to(device)


In [ ]:
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp

# Suppose `model` is your trained model and `data` is your input data
model.eval()

# Define GradCAM and GradCAM++
gradcam = GradCAM.from_config(model_type='resnet', arch=model, layer_name='layer4')
gradcam_pp = GradCAMpp.from_config(model_type='resnet', arch=model, layer_name='layer4')

# Get the GradCAM mask
mask, _ = gradcam(data)
mask_pp, _ = gradcam_pp(data)

# Visualize the GradCAM and GradCAM++ masks
heatmap, result = visualize_cam(mask, data)
heatmap_pp, result_pp = visualize_cam(mask_pp, data)

# Save results
cv2.imwrite('gradcam.jpg', result)
cv2.imwrite('gradcam_pp.jpg', result_pp)